 
1. Join data by student ID. 

In [ ]:
library(readxl)
library(dplyr)
library(tidyverse)
library(rattle)
library(rpart.plot)
library(RColorBrewer)
library(xgboost)
library(caret)
library(leaps)

fall_2011_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Fall 2011_SP.csv")
fall_2012_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Fall 2012_SP.csv")
fall_2013_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Fall 2013_SP.csv")
fall_2014_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Fall 2014_SP.csv")
fall_2015_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Fall 2015_SP.csv")
fall_2016_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Fall 2016_SP.csv")
Spring_2012_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Spring 2012_SP.csv")
Spring_2013_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Spring 2013_SP.csv")
Spring_2014_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Spring 2014_SP.csv")
Spring_2015_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Spring 2015_SP.csv")
Spring_2016_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Spring 2016_SP.csv")
Spring_2017_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Spring 2017_SP.csv")
Sum_2012_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Sum 2012.csv")
Sum_2013_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Sum 2013.csv")
Sum_2014_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Sum 2014.csv")
Sum_2015_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Sum 2015.csv")
Sum_2016_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Sum 2016.csv")
Sum_2017_SP <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Progress Data/Sum 2017.csv")
Fall_2011_ST <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Static Data/Fall 2011_ST.csv")
Fall_2012_ST <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Static Data/Fall 2012.csv")
Fall_2013_ST<- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Static Data/Fall 2013.csv")
Fall_2014_ST<- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Static Data/Fall 2014.csv")
Fall_2015_ST<- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Static Data/Fall 2015.csv")
Fall_2016_ST<- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Static Data/Fall 2016.csv")
Spring_2012_ST<- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Static Data/Spring 2012_ST.csv")
Spring_2013_ST<- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Static Data/Spring 2013.csv")
Spring_2014_ST<- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Static Data/Spring 2014.csv")
Spring_2015_ST<- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Static Data/Spring 2015.csv")
Spring_2016_ST<- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Static Data/Spring 2016.csv")
FASFA <- read_excel("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Student Financial Aid Data/2011-2017_Cohorts_Financial_Aid_and_Fafsa_Data.xlsx")
FASFA <- rename(FASFA, StudentID = `ID with leading`)
FASFA$StudentID <- as.integer(FASFA$StudentID)

FASFA$Loans <- rowSums(FASFA[, c("2012 Loan", "2013 Loan", "2014 Loan", "2015 Loan", "2016 Loan", "2017 Loan")], na.rm = TRUE)
FASFA$Scholarships <- rowSums(FASFA[, c("2012 Scholarship", "2013 Scholarship", "2014 Scholarship", "2015 Scholarship", "2016 Scholarship", "2017 Scholarship")], na.rm = TRUE)
FASFA$Income <- FASFA$`Adjusted Gross Income` + FASFA$`Parent Adjusted Gross Income`
FASFA$Grant <- rowSums(FASFA[, c("2012 Grant", "2013 Grant", "2014 Grant", "2015 Grant", "2016 Grant", "2017 Grant")], na.rm = TRUE)
FASFA$Workstudy <- rowSums(FASFA[, c("2012 Work/Study", "2013 Work/Study", "2014 Work/Study", "2015 Work/Study", "2016 Work/Study", "2017 Work/Study")], na.rm = TRUE)

FASFA$Income[is.na(FASFA$Income)] <- mean(FASFA$Income, na.rm = TRUE)

# Join the data by student ID
SP<- rbind(fall_2011_SP, fall_2012_SP, fall_2013_SP, fall_2014_SP, fall_2015_SP, fall_2016_SP, Spring_2012_SP, Spring_2013_SP, Spring_2014_SP, Spring_2015_SP, Spring_2016_SP, Spring_2017_SP, Sum_2012_SP, Sum_2013_SP, Sum_2014_SP,Sum_2015_SP, Sum_2016_SP, Sum_2017_SP)
ST<- rbind(Fall_2011_ST,Fall_2012_ST,Fall_2013_ST,Fall_2014_ST,Fall_2015_ST,Fall_2016_ST,Spring_2012_ST,Spring_2013_ST,Spring_2014_ST, Spring_2015_ST,Spring_2016_ST)

Student_ID <- inner_join(SP, ST, by="StudentID")
Student_ID <- inner_join(Student_ID, FASFA, by = "StudentID")

Student_ID$Race <- rowSums(Student_ID[, c("Hispanic", "AmericanIndian", "Asian", "Black", "NativeHawaiian", "White", "TwoOrMoreRace")], na.rm = TRUE)

2. Clean data and perform feature engineering.

In [ ]:
# check for missing values
colSums(is.na(Student_ID))

# remove columns with missing values
Student_ID <- Student_ID[, colSums(is.na(Student_ID)) == 0]

# Remove duplicates
Student_ID <- Student_ID[!duplicated(Student_ID$StudentID),]

# remove duplicates based on the "id" column
Student_ID <- distinct(Student_ID, StudentID, .keep_all = TRUE)

# identify columns with only 0 values
zero_cols <- sapply(Student_ID, function(x) all(x == 0))

# remove identified columns
Student_ID <- Student_ID[, !zero_cols]

complete.cases(Student_ID)
# print the result
Student_ID

3. Separate the data into training and test based on test IDs I have provided in testids.csv file. We will call the test dataset Kaggletest just to differentiate it from other test datasets we will create later in step 5. Set the kaggletest dataset aside until step 6.

In [ ]:
test_ids <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/Student Retention Challenge Data/Test Data/TestIDs.csv", header = TRUE)
test1 <- test_ids$StudentID
train1 <- Student_ID[!Student_ID$StudentID %in% test_ids$StudentID,]
Kaggletest <- Student_ID[Student_ID$StudentID %in% test_ids$StudentID,]

4. Add the Dropout labels column to the train dataset by joining to the data in droppoutTrainlabels.csv file.

In [ ]:
# read the dropoutTrainlabels.csv file
dropout_labels <- read.csv("/kaggle/input/finc-530-student-dropout-prediction-challenge/DropoutTrainLabels.csv")

# merge the train and dropout labels dataframes
train1 <- merge.data.frame(train1, dropout_labels, by = "StudentID")

5. Fit various models to the training dataset by going through the usual process of further splitting this dataset into train and test and measuring performance.

In [ ]:
# Convert Dropout to a factor variable if necessary
colnames(train1) <- make.names(colnames(train1))
train1 <- train1[, c("StudentID", "Cohort.x","AcademicYear", "Major1", "TermGPA", "CumGPA","RegistrationDate","NumColCredAttemptTransfer", "NumColCredAcceptTransfer", "Loans", "Scholarships","Grant","Income", "Dropout")]
train1$Dropout <- as.factor(train1$Dropout)
intrain <- createDataPartition(train1$Dropout, p = 0.5, list = FALSE)
test1 <- train1[-intrain,]
# Create a train control object for 10-fold cross validation
trctrl <- trainControl(method = "cv", number = 10, )

In [ ]:
# tree fit
tree_fit <- train(Dropout ~ ., data = train1, method = "rpart",
                  trControl=trctrl)

#To see the tuned complexity parameter (Gini Coeff)
tree_fit$bestTune
#To see the tree splits
tree_fit$finalModel
#Plot complexity parameter tuning runs
plot(tree_fit)
#Plot the tree

fancyRpartPlot(tree_fit$finalModel)
#Predict
predictions <- predict(tree_fit, newdata = test1)

#Performance metrics
#Calculate Mean Square Error (MSE)
mean(( predictions - test1$Dropout)^2)
#To see the importance of the variables
treeImp <- varImp(tree_fit, scale = TRUE)
treeImp
plot(treeImp)


In [ ]:
#Random Forest

#Fit the random forest (method = "rf"). Set importance = TRUE to have the variable importance calculated.
#Parameter mtry in the train function lets you set how many variables are considered at each split
forest_fit <- train(Dropout ~., data = train1, method = "rf",
                    trControl = trctrl, preProcess=c('scale', 'center'))

#To see model details
forest_fit
#To see the tuned mtry parameter.  Mtry is the number of randomly selected predictors
forest_fit$bestTune
#To see the the % variance explained
forest_fit$finalModel
#Plot complexity parameter tuning runs
plot(forest_fit)
#Predict
predictions <- predict(forest_fit, newdata = test1)

#Calculate MSE
mean(( predictions - test1$Dropout)^2)
#To see the importance of the variables
forestImp <- varImp(forest_fit)
forestImp
plot(forestImp)

In [ ]:
# Convert target to numeric
tr_label <- as.numeric(train1$Dropout) - 1
ts_label <- as.numeric(test1$Dropout) - 1

# One hot encode predictor variables to convert categorical variables to numeric
m_train <- model.matrix(~.+0, data = train1[, -5]) 
m_test <- model.matrix(~.+0, data = test1[, -5])

dtrain <- xgb.DMatrix(data = m_train, label = tr_label) 
dtest <- xgb.DMatrix(data = m_test, label = ts_label)

params <- list(
  booster = "gbtree",
  objective = "multi:softmax",
  num_class = 3,
  eta = 0.3,
  gamma = 0,
  max_depth = 2,
  min_child_weight = 1, 
  subsample = 1,
  colsample_bytree = 1
)

xgbcv <- xgb.cv(
  params = params,
  data = dtrain,
  nrounds = 1000, 
  nfold = 5,
  showsd = T,
  stratified = T, 
  print_every_n = 10,
  early_stop_rounds = 20, 
  maximize = F
)

bst <- xgb.train(params = params, data = dtrain, nrounds = 100)

pred <- predict(bst, dtest)
pred <- ifelse(pred > 0.5, 1, 0)
pred <- as.factor(pred)

ts_label <- as.factor(ts_label)

confusionMatrix(pred, ts_label)

mat <- xgb.importance(feature_names = colnames(m_train), model = bst)
xgb.plot.importance(importance_matrix = mat)


In [ ]:
#Bagging
#No tuning parameters supported
bag_fit <- train(Dropout ~., data = train1, method = "treebag",
                    trControl=trctrl)
bag_fit
predictions <- predict(bag_fit, newdata = test1)

mean(( predictions - test1$Dropout)^2)
#To see the importance of the variables
bagImp <- varImp(bag_fit, scale=TRUE)
bagImp
plot(bagImp)

In [ ]:
#Fit the SVM model to training data
#svmRadial uses the Radial Kernel.  
#You can explicitly specify the cost parameter by using the C = option 
#and the gamma (sigma) parameter using the sigma = option
modSVMFit <- train(Dropout ~ .,method="svmRadial",sigma =.2,trControl=trctrl,data=train1)
#See model fit details
modSVMFit$finalModel
#See the tuning parametrs used (cost C, and sigma of the radial kernel function)
modSVMFit$bestTune
#See the results details by each optimization run
modSVMFit$results
#Predict test dataset
SVMpredict <- predict(modSVMFit,test1)

confusionMatrix(SVMpredict,test1$Dropout)

6. Predict using the Kaggletest dataset. This will result in a column with 1s and 0s in the prediction object you create.

In [ ]:
# Predict using the Kaggletest dataset
predictions <- predict(forest_fit, newdata = Kaggletest)

predictions_binary <- as.integer(predictions == levels(predictions)[2])

predictions_binary

7. Combine this prediction column with the student ID column in the kaggletest dataframe and create a new data frame that has two columns Student ID and Dropout

In [ ]:
nrow(Kaggletest)
# create a new dataframe with Student ID and Dropout columns
new_df <- data.frame(StudentID = Kaggletest$StudentID, Dropout = predictions_binary)

8. Write the dataframe created above to a csv file and submit to Kaggle. Note: the submission file should look exactly like the sample submission file I have provided. Once you submit to Kaggle you will get the accuracy (F1 measure) that determines your position on the leaderboard.



In [ ]:
# write the result dataframe to a csv file
write.csv(new_df, "submission.csv", row.names = FALSE)